## Restart after running this cell!

You must run this cell and then restart and rerun everything for the PyTorch version to be correct. Otherwise the model will run but not produce any meaningful output.

In [1]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

! pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

CUDA version: 10.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
!pip install big-sleep --upgrade

Requirement already up-to-date: big-sleep in /usr/local/lib/python3.6/dist-packages (0.3.1)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
from IPython.display import Image, display
import string
import torch
from torchvision.utils import save_image
import numpy as np

from big_sleep import Imagine
from big_sleep.clip import tokenize

from nltk.corpus import stopwords

from skimage.measure import compare_ssim

import cv2
from pathlib import Path

import PIL
from PIL import ImageFont, ImageDraw

TEXT = 'story_hallucinator' 
SAVE_EVERY = 1
SAVE_PROGRESS = True
LEARNING_RATE = 0.1
ITERATIONS =  1

def train_step(self, epoch, i, rand=0):
  total_loss = 0

  for _ in range(self.gradient_accumulate_every):
      losses = self.model(self.encoded_text) 
      loss = (sum(losses) / self.gradient_accumulate_every) + rand*np.random.randn()
      total_loss += loss
      loss.backward()

  self.optimizer.step()
  self.optimizer.zero_grad()

  if (i + 1) % self.save_every == 0:
      with torch.no_grad():
          # best = torch.topk(losses[2], k = 1, largest = False)[1]
          mres = self.model.model()
          image = mres[len(mres)-1].cpu()
          num = i // self.save_every
          save_image(image, Path(f'./{self.textpath}.{num}.png'))

model = Imagine(
    text = TEXT,
    save_every = SAVE_EVERY,
    lr = LEARNING_RATE,
    iterations = ITERATIONS,
    save_progress = SAVE_PROGRESS
)
filename = TEXT.replace(' ', '_')
### from https://a.ttent.io/n by Terise Cruven
# all_text = """""Out early for drugs and octopus," Ezzy texted. She was sitting on the hallway floor of the Institute's distribution center, head leaning back against the wall. Three seconds after 6 AM she started tapping impatiently on the glass between her and the narcotics cabinet. She handed the requisition to the half-awake clerk, and he pushed the bottles across the counter with a detached shrug. She dropped the bottles into her backpack and slipped down the stairwell to the supply tunnel that ran under campus to the marine center. She waited on the dock until 9 when the divemaster finally brought her two harp-toothed octopuses. """

## from Paul Clifford
# all_text = """It was a dark and stormy night; the rain fell in torrents, except at occasional intervals, when it was checked by a violent gust of wind which swept up the streets (for it is in London that our scene lies), rattling along the house-tops, and fiercely agitating the scanty flame of the lamps that struggled against the darkness. Through one of the obscurest quarters of London, and among haunts little loved by the gentlemen of the police, a man, evidently of the lowest orders, was wending his solitary way."""

### from Metamorphoses by Fanz Kafka
all_text = """One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked."""


burnin=10 #
checkin_gap = 10
long_sim_gap = 10
span = 6
iterations = 101
display_gap = 10
similarity = 0.8

words = all_text.split()
all_text_list = [" ".join(words[i:i+span]) for i in range(0, len(words), span)]
### Wordsworth
# all_text_list = ["""I wandered lonely as a cloud,""",
# """That floats on high over vales and hills.""",
# """When all at once I saw a crowd,""",
# """A host, of golden daffodils;""",
# """Beside the lake, beneath the trees,""",
# """Fluttering and dancing in the breeze.""",
# """Continuous as the stars that shine""",
# """And twinkle on the milky way,""",
# """They stretched in never-ending line""",
# """Along the margin of a bay:""",
# """Ten thousand saw I at a glance,""",
# """Tossing their heads in sprightly dance.""",
# """The waves beside them danced; but they""",
# """Out-did the sparkling waves in glee:""",
# """A poet could not but be gay,""",
# """In such a jocund company:""",
# """I gazed—and gazed—but little thought""",
# """What wealth the show to me had brought"""]
all_text_full = all_text_list
# all_text_list = [""""early for drugs and octopus""",
#                  """"sitting on hallway floor""",
#                  """Institute distribution center,""",
#                  """head leaning back against wall.""",
#                  """Three seconds after 6 AM""",
#                  """glass narcotics cabinet.""",
#                  """requisition half awake clerk,""",
#                  """bottles across the counter.""",
#                  """bottles into her backpack""",
#                  """slipped down the stairwell""",
#                  """supply tunnel to the marine center.""",
#                  """waited on the dock""",
#                  """divemaster brought her""",
#                  """two harp toothed octopuses. """
#                  ]
# all_text_full = [""""Out early for drugs and octopus," Ezzy texted.""",
#                  """"She was sitting on the hallway floor""",
#                  """of the Institute's distribution center,""",
#                  """head leaning back against the wall.""",
#                  """Three seconds after 6 AM she started tapping impatiently""",
#                  """on the glass between her and the narcotics cabinet.""",
#                  """She handed the requisition to the half-awake clerk,""",
#                  """and he pushed the bottles across the counter with a detached shrug.""",
#                  """She dropped the bottles into her backpack""",
#                  """and slipped down the stairwell to the """,
#                  """supply tunnel that ran under campus to the marine center.""",
#                  """She waited on the dock until 9""",
#                  """when the divemaster finally brought her""",
#                  """two harp-toothed octopuses. """
#                  ]
iter_num = 0
last_one = 0
rand = 0
model.text = all_text_list[0].translate(str.maketrans('', '', string.punctuation))
model.encoded_text = tokenize(model.text).cuda()
for j in range(burnin):
    train_step(model, 0, 0, rand)
for epoch in range(len(all_text_list)):
    print(all_text_list[epoch])
    restart_point = iter_num
    i = 0
    while i < iterations:
        model.text = all_text_list[epoch].translate(str.maketrans('', '', string.punctuation))
        model.encoded_text = tokenize(model.text).cuda()
        train_step(model, epoch, iter_num, rand)
        
        if i % display_gap == 0 or i == checkin_gap or i % long_sim_gap == 0:
          print(f'iter: {iter_num} text={all_text_list[epoch]}')
          image_cur = Image(f'./{filename}.{iter_num}.png')
          display(image_cur)
        
        if i % checkin_gap == 0 and i > 0:
          imageA = cv2.imread(f'./{filename}.{iter_num}.png')
          imageB = cv2.imread(f'./{filename}.{restart_point}.png')
          # convert the images to grayscale
          grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
          grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
          (score, diff) = compare_ssim(grayA, grayB, full=True)
          toinc = checkin_gap
          print(f'iter{iter_num}: rand={rand} sim={score}')
          if score>similarity:
              print(f'restart!')
              model = Imagine(
                  text = TEXT,
                  save_every = SAVE_EVERY,
                  lr = LEARNING_RATE,
                  iterations = ITERATIONS,
                  save_progress = SAVE_PROGRESS
              )
              model.text = all_text_list[epoch].translate(str.maketrans('', '', string.punctuation))
              model.encoded_text = tokenize(model.text).cuda()
              for j in range(burnin):
                train_step(model, epoch, iter_num, rand) 
              iter_num = restart_point
              i = 0
              rand = 0
              continue
        # if i % long_sim_gap == 0 and i > 0:
        #   imageA = cv2.imread(f'./{filename}.{iter_num}.png')
        #   imageB = cv2.imread(f'./{filename}.{iter_num-long_sim_gap}.png')
        #   # convert the images to grayscale
        #   grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
        #   grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
        #   (score, diff) = compare_ssim(grayA, grayB, full=True)
        #   print(f'iter{iter_num}: rand={rand} sim={score}')
        #   if score>similarity:
        #     print(f'randomize!')
        #     rand = rand + 0.5
        #   else:
        #     rand= rand *0.9
        i += 1
        iter_num += 1
        
        
    for i in range(last_one,iter_num):
      msg = all_text_full[epoch]
      img = PIL.Image.open(f'./{filename}.{i}.png')
      W, H = img.size
      draw = ImageDraw.Draw(img)
      w, h = draw.textsize(msg)
      x, y = (W-w)/2, 7*(H-h)/8
      adj = 1
      #move right
      shadowColor = "white"
      draw.text((x-adj, y), msg, fill=shadowColor)
      #move left
      draw.text((x+adj, y), msg, fill=shadowColor)
      #move up
      draw.text((x, y+adj), msg, fill=shadowColor)
      #move down
      draw.text((x, y-adj), msg, fill=shadowColor)
      #diagnal left up
      draw.text((x-adj, y+adj), msg, fill=shadowColor)
      #diagnal right up
      draw.text((x+adj, y+adj), msg, fill=shadowColor)
      #diagnal left down
      draw.text((x-adj, y-adj), msg, fill=shadowColor)
      #diagnal right down
      draw.text((x+adj, y-adj), msg, fill=shadowColor)
      draw.text(((W-w)/2,7*(H-h)/8), msg, fill="black")
      # font = ImageFont.load_default()#.truetype("sans-serif.ttf", 16)
      # draw.text((0, 0),,(255,255,255),font=font)
      img.save(f'./{filename}.{i}.png')
    last_one = iter_num



Output hidden; open in https://colab.research.google.com to view.

In [26]:
!zip archive.zip *.png

updating: story_hallucinator.0.png (deflated 0%)
updating: story_hallucinator.1000.png (deflated 0%)
updating: story_hallucinator.1001.png (deflated 0%)
updating: story_hallucinator.1002.png (deflated 0%)
updating: story_hallucinator.1003.png (deflated 0%)
updating: story_hallucinator.1004.png (deflated 0%)
updating: story_hallucinator.1005.png (deflated 0%)
updating: story_hallucinator.1006.png (deflated 0%)
updating: story_hallucinator.1007.png (deflated 0%)
updating: story_hallucinator.1008.png (deflated 0%)
updating: story_hallucinator.1009.png (deflated 0%)
updating: story_hallucinator.100.png (deflated 0%)
updating: story_hallucinator.1010.png (deflated 0%)
updating: story_hallucinator.1011.png (deflated 0%)
updating: story_hallucinator.1012.png (deflated 0%)
updating: story_hallucinator.1013.png (deflated 0%)
updating: story_hallucinator.1014.png (deflated 0%)
updating: story_hallucinator.1015.png (deflated 0%)
updating: story_hallucinator.1016.png (deflated 0%)
updating: story_

In [27]:
!cp archive.zip /content/drive/MyDrive/story_hallucinator

Next download the zip file from Drive, unzip it in a folder and run the following:

```
ffmpeg -framerate 10 -i story_hallucinator.%d.png -c:v libx264 -crf 0 story_hallucination.mp4
```



In [19]:
!rm story_*

In [20]:
!ls -trl archive.zip

-rw-r--r-- 1 root root 608582237 Jan 22 07:46 archive.zip


In [21]:
!rm archive.zip